**PYTORCH IMPLEMENTATION OF MNIST DATASET TRAINING USING CPU ONLY**

In [39]:
#Importing the necessary libraries and modules
import torch
import torch.nn as nn
import torch.optim as optim

In [40]:
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
device = torch.device('cpu') #Specify the device to train the dataset
print("Using device: ", device)

Using device:  cpu


In [41]:
transform = transforms.ToTensor() #Transforms the data to Tensors (the data points of the image pixels)
train_data = datasets.MNIST(root="./data", train = True, download = True, transform = transform)
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)

In [43]:
#Defining the model network architecture
class Net(nn.Module):

  def __init__(self):
    super().__init__()
    self.fc = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 256),
        nn.ReLU(),
        nn.Linear(256, 10),
    )

  def forward(self, x):
    return self.fc(x)

In [44]:
model = Net().to(device) #Specify the device where the model gets stores
criterion = nn.CrossEntropyLoss() #Loss function
optimizer = optim.Adam(model.parameters(), lr=0.001) #Optimizer function


In [46]:
from tqdm import tqdm #Progress bar library
import time

In [48]:
Epochs = 5
print(f"Training the model using the {device}")
start = time.time()
for epoch in range(Epochs):
  loop = tqdm(train_loader,desc = "Training Epoch {}".format(epoch))
  for x, y in loop:
    x, y = x.to(device), y.to(device)

    optimizer.zero_grad() #Gradient reset for smooth propagation
    preds= model(x) #Forward pass
    loss = criterion(preds, y)
    loss.backward()  #Backward pass
    optimizer.step()  #Updates the weights

    batch_acc = (preds.argmax(dim=1) == y).float().mean().item() #Batch accuracy

    loop.set_postfix({"loss": f"{loss.item():.4f}",
                     "acc": f"{batch_acc*100:.2f}%)",
                     "lr" : optimizer.param_groups[0]["lr"]})

end = time.time()
print(f"\n Time taken to train the model: {end-start} seconds")

Training the model using the cpu


Training Epoch 4: 100%|██████████| 938/938 [00:11<00:00, 79.54it/s, loss=0.0074, acc=100.00%), lr=0.001]


 Time taken to train the model: 59.08636665344238 seconds


TRAINING THE MNIST DATASET USING CUDA accelerated COMPUTING

In [49]:
device_cuda = torch.device("cuda")
print(f"Using device: {device_cuda}")

Using device: cuda


In [50]:
#Reinitializing the model with CUDA support
model_cuda_accelerated = Net().to(device_cuda)
optimizer_cuda = optim.Adam(model_cuda_accelerated.parameters(), lr=0.001)

In [52]:
Epochs = 5
print(f"Training the model using the {device_cuda}")
start = time.time()
for epoch in range(Epochs):
  loop = tqdm(train_loader,desc = "Training Epoch {}".format(epoch))
  for x, y in loop:
    x, y = x.to(device_cuda), y.to(device_cuda)

    optimizer_cuda.zero_grad() #Resets the gradients of the model to ensure that the gradients are not accumulated
    preds= model_cuda_accelerated(x) #forwards pass
    loss = criterion(preds, y)
    loss.backward() #Backward pass based on the loss
    optimizer_cuda.step() #Updates the weights

    batch_acc = (preds.argmax(dim=1) == y).float().mean().item() #Batch accuracy

    loop.set_postfix({"loss": f"{loss.item():.4f}",
                     "acc": f"{batch_acc*100:.2f}%)",
                     "lr" : optimizer.param_groups[0]["lr"]})

end = time.time()
print(f"\n Time taken to train the model: {end-start} seconds")

Training the model using the cuda


Training Epoch 4: 100%|██████████| 938/938 [00:09<00:00, 98.04it/s, loss=0.0012, acc=100.00%), lr=0.001]


 Time taken to train the model: 48.60098624229431 seconds
